In [2]:
%load_ext autoreload
%autoreload 2

---

In [3]:
import pandas as pd

df = pd.read_feather('../data/processed/twitter_tweets_processed.feather')
df.head(4)

,url,date,rawContent
0,https://twitter.com/YueDongCS/status/164159107...,2023-03-30 23:59:46,"[condolence, sad, nlp, ai, community, lose, br..."
1,https://twitter.com/Mlearning_ai/status/164159...,2023-03-30 23:59:43,"[hiring, top, job, future, fueled, generative,..."
2,https://twitter.com/HackerAran7/status/1641591...,2023-03-30 23:59:43,"[hack, stem, science, stemeducation, education..."
3,https://twitter.com/Stemble_/status/1641590942...,2023-03-30 23:59:14,"[mark, calendar, apple, enthusiast, june, 5, a..."


---